In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

In [2]:
plate_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1.csv"
plate_1_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1_repeat.csv"
plate_2_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_1.csv"
plate_2_2 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_2.csv"
plate_2_3 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_3.csv"
plate_2_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat.csv"
plate_2_repeat_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat _96.csv"

In [3]:
def read_in_plate(csv_file):
    
    """ Iterates over the raw data file to find the line numbers at which the metadata table and raw data table begin for 
    each channel and reapeat. Calculates the length of those tables. Finds the G-factor. Determines the format of data (plate 1 or 2).
    
    :param csv_file: Raw data file in csv format.
    :param type: str
    :return: A tuple containing a list of tuples (one tuple for each channel) and a string representing data format. """
    
    with open(csv_file) as file:
        all_data_lines = list(csv.reader(file, delimiter=','))   # read the csv file and cast it into a list containing all lines
        
    blank_indices = list(index for index, item in enumerate(all_data_lines) if item == [])   # list containing indices of all blank rows
    blanks = np.array(blank_indices)
    values = []   # list for storage of tuples

    for index, item in enumerate(all_data_lines):   # iterate over each line of the csv file
        if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) == None and re.findall(r"Formula", all_data_lines[index+1][10]) != ['Formula']:
            skiprows = index + 9   # Set the skiprows parameter for raw data table
            skiprows_meta = index + 1   # Set the skiprows parameter for metadata table
            end_of_data = blanks[blanks > skiprows].min()   # calculate the end of data table by finding the smallest blank index after the beginning of data table
            values.append((skiprows, end_of_data-skiprows+1, skiprows_meta))   # add the skiprows, caculated number of data lines and skiprows for metadata parameters to the list as a tuple
            data_format = 'plate1'
        
        if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) != None:
            skiprows = index + 10
            skiprows_meta = index + 1
            end_of_data = blanks[blanks > skiprows].min()
            values.append((skiprows, end_of_data-skiprows, skiprows_meta))
            data_format = 'plate2'

        if item != [] and len(item) > 1 and re.findall(r"G-factor", item[0]) == ["G-factor"]:
            g_factor = float(item[4])   
                
    return values, data_format

In [4]:
p2 = read_in_plate(plate_2_repeat)

In [5]:
p2

([(38, 17, 29), (66, 17, 57), (122, 17, 113), (150, 17, 141)], 'plate2')

In [5]:
def process_plate(csv_file, values, wells):    
    
    """ Iterates over the raw data file and creates data frames for the data and metadata for each channel, converts them into
    a 384 or 96 by 1 format and adds them into a dictionary.
    
    :param csv file: Raw data file in csv format.
    :param type: str
    :param values: A list containg tuples with read in parameters for each channel and the data format parameter.
    :param type: tuple
    :param wells: Number of wells on the plate.
    :param type: str
    :return: A dictionary containg a dictionary for each repeat containg the metadata df and a dictionary with s and p channel dfs. """
    
    plate_dimensions = {'96':(8, 12), '384':(16, 24)}   

    row_letters = list(string.ascii_uppercase)[0:plate_dimensions[wells][0]]   # generate letters for the data table
    col_numbers = list(np.arange(1, plate_dimensions[wells][1]+1).astype(str))   # generate numbers for the data table
    well_ids = ['%s%s' % (item[0], item[1]) for item in product(row_letters, col_numbers)]   # generate well IDs for the data table

    data_frames = {}   # dictionary to store the data frames
    counter = 1   # counter to enable alternating labelling of data frames as p or s
    
    for index,item in enumerate(values[0]):   # iterate over each tuple in the list
        
        if values[1] == 'plate1':   # raw data table does not have row and column names so 'names' parameter passed to omit the last column
            raw_data = pd.read_csv(csv_file, sep=',', names=col_numbers, index_col=False, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')
        
        if values[1] == 'plate2':   # raw data table has row an column names, so 'index_col' must be 0 
            raw_data = pd.read_csv(csv_file, sep=',', index_col=0, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')
            raw_data.drop(raw_data.columns[-1], axis=1, inplace=True)   # delete the last column because it is empty

        # generate df for metadata (noumber of rows is always 1) and conver measurement time into datetime object   
        metadata = pd.read_csv(csv_file, sep=',', engine='python', skiprows=item[2], nrows=1, encoding='utf-8').astype({'Measurement date': 'datetime64[ns]'})
        data_to_array = np.reshape(raw_data.to_numpy(), (int(wells), 1))   # convert data frames to numpy arrays and reshape into 1D array

        if counter % 1 == 0: 
            new_data = pd.DataFrame(data=data_to_array, index=well_ids, columns=['p'])   # generate new 384 (or 96) by 1 data frame with p channel data
            data_frames[f'repeat_{int(counter)}'] = {'metadata':metadata, 'data': {'p': new_data, 's':''}}   # add data and metadata dfs to the dictionary
        
        if counter % 1 != 0:
            new_data = pd.DataFrame(data=data_to_array, index=well_ids, columns=['s'])   # generate new 384 (or 96) by 1 data frame with s channel data
            data_frames[f'repeat_{int(counter-0.5)}']['data']['s'] = new_data

        counter = counter + 0.5
        
    return data_frames

In [6]:
p2r = process_plate(plate_2_repeat, p2, '384')

In [11]:
p2r['repeat_1']['metadata']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Group,Label,ScanX,ScanY,Measinfo,Kinetics,Measurement date,Unnamed: 17
0,1,1,NaN,14.4,18.98,18.8,61.7,61.5,18.98,18.9,1,Copy of Kris FP Fluorescein anisotropy(1),0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),0,2020-11-17 13:33:43,NaN


In [12]:
p2r['repeat_2']['metadata']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Group,Label,ScanX,ScanY,Measinfo,Kinetics,Measurement date,Unnamed: 17
0,1,2,NaN,14.4,18.8,18.7,61.5,61.8,18.9,19.1,1,Copy of Kris FP Fluorescein anisotropy(1),0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),0,2020-11-17 13:35:26,NaN


In [33]:
p2r['repeat_1']['data']['p']

,p
A1,18781491.0
A2,27318000.0
A3,15885300.0
A4,21036054.0
A5,22823211.0
...,...
P20,NaN
P21,NaN
P22,NaN
P23,NaN


In [34]:
p2r['repeat_1']['data']['s']

,s
A1,20469296.0
A2,29296716.0
A3,18210982.0
A4,23159988.0
A5,24960618.0
...,...
P20,NaN
P21,NaN
P22,NaN
P23,NaN


In [35]:
p2r['repeat_2']['data']['p']

,p
A1,18964719.0
A2,27435568.0
A3,16237095.0
A4,21291729.0
A5,22591167.0
...,...
P20,NaN
P21,NaN
P22,NaN
P23,NaN
